In [1]:
# Author and date
import datetime, getpass
print(getpass.getuser(), datetime.datetime.today())

import datajoint as dj
print('DataJoint Version', dj.__version__)

dimitri 2019-11-08 06:25:01.356395
DataJoint Version 0.12.1


# Projection

The projection operator `proj` selects or renames attributes or calculates new attributes. 

Let's create a table `Rectangle` and fill it with random rectangles.

In [3]:
schema = dj.schema('test_proj')

In [4]:
@schema
class Rectangle(dj.Manual):
    definition = """
    rectangle : int
    ---
    width : smallint unsigned
    height : smallint unsigned
    color_r: tinyint unsigned
    color_g: tinyint unsigned
    color_b: tinyint unsigned 
    """

In [5]:
import random
for i in range(100):
    Rectangle.insert1((
        i, 
        random.randint(0, 100), 
        random.randint(0, 100), 
        random.randint(0,255),
        random.randint(0,255),
        random.randint(0,255)
    ))

In [6]:
Rectangle()

rectangle,width,height,color_r,color_g,color_b
0,46,20,238,214,212
1,56,2,106,82,53
2,28,34,159,228,96
3,86,58,136,168,22
4,35,59,17,211,226
5,59,71,230,193,82
6,7,22,114,31,158
7,54,51,132,81,177
8,99,53,71,196,207
9,100,22,95,213,54


You can use `proj` to select to select a subset of attributes:

In [7]:
Rectangle.proj('width', 'height')

rectangle,width,height
0,46,20
1,56,2
2,28,34
3,86,58
4,35,59
5,59,71
6,7,22
7,54,51
8,99,53
9,100,22


The primary attributes are always included. Projection cannot exclude the primary key. With no attributes specified, projection will return just the primary attributes:

In [8]:
Rectangle.proj()

rectangle
0
1
2
3
4
5
6
7
8
9


## Renaming
You can rename attributes by specifying the new names as keyword arguments.

In [9]:
Rectangle.proj(rec='rectangle', green='color_g', red='color_r', blue='color_b')

rec,red,green,blue
0,238,214,212
1,106,82,53
2,159,228,96
3,136,168,22
4,17,211,226
5,230,193,82
6,114,31,158
7,132,81,177
8,71,196,207
9,95,213,54


The ellipsis `...` allows including the remaining attributes that have not been renamed.

In [10]:
Rectangle.proj(..., green='color_g', red='color_r', blue='color_b')

rectangle,width,height,red,green,blue
0,46,20,238,214,212
1,56,2,106,82,53
2,28,34,159,228,96
3,86,58,136,168,22
4,35,59,17,211,226
5,59,71,230,193,82
6,7,22,114,31,158
7,54,51,132,81,177
8,99,53,71,196,207
9,100,22,95,213,54


## Exclusions

Prefixing attribute names with a dash `-` excludes them from the result. This can be used only together with the ellipsis.

In [11]:
# exclude height
Rectangle.proj(..., '-height')

rectangle,width,color_r,color_g,color_b
0,46,238,214,212
1,56,106,82,53
2,28,159,228,96
3,86,136,168,22
4,35,17,211,226
5,59,230,193,82
6,7,114,31,158
7,54,132,81,177
8,99,71,196,207
9,100,95,213,54


## Calculations
Finally, `proj` can add calculated attributes:

In [12]:
# Calculate rectangle properties
Rectangle.proj(
    area='width * height', 
    aspect = 'width / height',
    diagonal='sqrt(width * width + height * height)',
    brightness='round((color_r + color_g + color_b)/3)')

rectangle,area calculated attribute,aspect calculated attribute,diagonal calculated attribute,brightness calculated attribute
0,920,2.3000,50.15974481593781,221
1,112,28.0000,56.0357029044876,80
2,952,0.8235,44.04543109109048,161
3,4988,1.4828,103.73041983911952,109
4,2065,0.5932,68.60029154456998,151
5,4189,0.8310,92.31467922275417,168
6,154,0.3182,23.08679276123039,101
7,2754,1.0588,74.27651041883968,130
8,5247,1.8679,112.29425630903836,158
9,2200,4.5455,102.39140588936164,121


Calculated expressions use SQL syntax, including all SQL functions.

## Another example

Let's create the table person and demonstrate some projections.

In [13]:
@schema
class Person(dj.Manual):
    definition = """
    username : varchar(24)
    ---
    first_name : varchar(30)
    last_name : varchar(30)
    date_of_birth : date    
    """

In [14]:
Person.insert((
    ('johnnyd', 'John', 'Doe', '1967-01-23'),
    ('jane', 'Jane', 'Doe', '1984-02-29')
))

In [15]:
Person()

username,first_name,last_name,date_of_birth
jane,Jane,Doe,1984-02-29
johnnyd,John,Doe,1967-01-23


In [16]:
Person.proj(
    user='username',
    full_name="concat(last_name, ', ', first_name)", 
    age="FLOOR(DATEDIFF(NOW(), date_of_birth)/365.25)")

user,full_name calculated attribute,age calculated attribute
jane,"Doe, Jane",35
johnnyd,"Doe, John",52


## Cleanup
remove the schema

In [17]:
schema.drop()

Proceed to delete entire schema `test_proj`? [yes, No]: yes
